# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.233541813733                   -0.50    8.0         
  2   -7.249781453027       -1.79       -1.37    1.0   9.23ms
  3   -7.250261673148       -3.32       -1.65    2.0   9.94ms
  4   -7.250994562966       -3.13       -1.90    1.0   9.09ms
  5   -7.251324646187       -3.48       -2.64    1.0   9.59ms
  6   -7.251337655540       -4.89       -2.98    2.0   11.1ms
  7   -7.251338691780       -5.98       -3.58    2.0   10.8ms
  8   -7.251338780157       -7.05       -3.94    2.0   12.3ms
  9   -7.251338796895       -7.78       -4.38    2.0   11.8ms
 10   -7.251338798548       -8.78       -5.01    1.0   10.1ms
 11   -7.251338798639      -10.04       -5.14    3.0   12.3ms
 12   -7.251338798696      -10.25       -5.63    1.0   9.42ms
 13   -7.251338798704      -11.11       -6.30    3.0   11.5ms
 14   -7.251338798704      -12.23       -6.58    3.0   55.3ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.08232702715607189
[ Info: Arnoldi iteration step 2: normres = 0.5346692165605208
[ Info: Arnoldi iteration step 3: normres = 0.7806980330429258
[ Info: Arnoldi iteration step 4: normres = 0.2202056365273654
[ Info: Arnoldi iteration step 5: normres = 0.5874815243803898
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (2.39e-02, 6.91e-02, 5.59e-01, 1.66e-01, 1.08e-03)
[ Info: Arnoldi iteration step 6: normres = 0.22095147174548135
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (3.91e-03, 6.32e-02, 1.52e-01, 1.06e-01, 5.84e-02)
[ Info: Arnoldi iteration step 7: normres = 0.09855927741867178
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.87e-04, 6.06e-03, 1.39e-02, 6.95e-02, 5.91e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10422917788841543
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (8.39e-